In [ ]:
from transformers import BertTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
import torch
import torch.nn.utils.prune as prune
from torch.nn.functional import cross_entropy
from pyswarm import pso

# Load IMDb dataset
dataset = load_dataset('imdb')

# Use a small subset of the dataset for faster experimentation
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(1000))

# Load TinyBERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=2)

# Make all model parameters contiguous
for param in model.parameters():
    param.data = param.data.contiguous()

# Tokenize data with fixed sequence length
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

tokenized_train = small_train_dataset.map(tokenize_function, batched=True)
tokenized_test = small_test_dataset.map(tokenize_function, batched=True)

# Remove columns that are not tensors
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_test = tokenized_test.remove_columns(["text"])

# Define compute metrics function with additional metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Convert logits and labels to tensors to compute cross-entropy loss
    logits_tensor = torch.tensor(logits)
    labels_tensor = torch.tensor(labels)

    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, average="binary")
    f1 = f1_score(labels, predictions, average="binary")
    loss = cross_entropy(logits_tensor, labels_tensor).item()  # Cross-entropy loss calculation

    return {
        "accuracy": accuracy,
        "recall": recall,
        "f1": f1,
        "loss": loss
    }

# Define PSO fitness function (objective function)
def fitness_function(hyperparams):
    lr, batch_size, epochs = hyperparams
    batch_size = int(batch_size)
    epochs = int(epochs)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        learning_rate=lr,
        evaluation_strategy="steps",
        eval_steps=50,
        save_steps=50,
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        compute_metrics=compute_metrics
    )

    for param in model.parameters():
        param.data = param.data.contiguous()

    trainer.train()
    eval_results = trainer.evaluate()
    return -eval_results['eval_accuracy']  # Minimize negative accuracy for PSO

# PSO to optimize hyperparameters
lb = [1e-5, 8, 1]
ub = [1e-3, 16, 2]
best_hyperparams, _ = pso(fitness_function, lb, ub, swarmsize=5, maxiter=3)

print("Best hyperparameters:", best_hyperparams)
final_lr, final_batch_size, final_epochs = best_hyperparams
final_batch_size = int(final_batch_size)
final_epochs = int(final_epochs)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=final_epochs,
    per_device_train_batch_size=final_batch_size,
    learning_rate=final_lr,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

for param in model.parameters():
    param.data = param.data.contiguous()

# Apply structured pruning on entire channels of linear layers
def apply_structured_pruning(model, amount=0.4):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=0)
            prune.remove(module, 'weight')

# Apply structured pruning with 40% sparsity
apply_structured_pruning(model, amount=0.4)

# Train the final pruned model and evaluate with added metrics
trainer.train()
final_results = trainer.evaluate()
print("Final evaluation after structured pruning:", final_results)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Recall,F1
50,0.689700,0.667675,0.628000,0.627049,0.621951


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Recall,F1
50,0.627400,0.671813,0.631000,0.875000,0.698283
100,0.541400,0.610344,0.686000,0.735656,0.695736


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Step,Training Loss,Validation Loss,Accuracy,Recall,F1
50,0.006600,5.766375,0.689000,0.485656,0.603822


Stopping search: maximum iterations reached --> 3
Best hyperparameters: [7.10991852e-04 8.16467595e+00 1.96990985e+00]


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,No log,1.549414,0.742000,0.758197,0.741483


Final evaluation after structured pruning: {'eval_loss': 1.549413800239563, 'eval_accuracy': 0.742, 'eval_recall': 0.7581967213114754, 'eval_f1': 0.7414829659318637, 'eval_runtime': 0.689, 'eval_samples_per_second': 1451.29, 'eval_steps_per_second': 181.411, 'epoch': 1.0}


In [ ]:
# Define the directory where the model and tokenizer will be saved
output_dir = "./saved_model"

# Save the model and tokenizer after training
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Optionally, you can also save the training arguments and optimizer states if needed
trainer.save_model(output_dir)  # This will save the model, tokenizer, and additional components